In [ ]:
#https://github.com/BrunoKrinski/PytorchClassification

In [ ]:
!wget https://www.inf.ufpr.br/bakrinski/butterflies_features.zip
!unzip butterflies_features.zip

In [ ]:
!wget https://www.inf.ufpr.br/bakrinski/sports_features.zip
!unzip sports_features.zip

In [ ]:
import os
import json
import time
import glob
import argparse
import numpy as np
import torch.nn as nn
import torch, torchvision
import torch.optim as optim
import matplotlib.pyplot as plt

from PIL import Image
from tqdm import tqdm
from torchsummary import summary
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms

from torchvision.models import vgg16, VGG16_Weights
from torchvision.models import resnet152, ResNet152_Weights
from torchvision.models import resnet50, ResNet50_Weights

In [ ]:
#model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
#print(model)
#summary(model, input_size=(3, 224, 224), batch_size=1, device='cpu')

In [ ]:
#model = resnet152(weights=ResNet152_Weights.IMAGENET1K_V1)
#print(model)
#summary(model, input_size=(3, 224, 224), batch_size=1, device='cpu')

In [ ]:
model = vgg16(weights=VGG16_Weights.IMAGENET1K_V1)
print(model)
summary(model, input_size=(3, 224, 224), batch_size=1, device='cpu')

In [ ]:
size = 224

image_transforms = { 
  'test': transforms.Compose([
      transforms.Resize(size=size),
      transforms.CenterCrop(size=size),
      transforms.ToTensor(),
      transforms.Normalize([0.485, 0.456, 0.406],
                           [0.229, 0.224, 0.225])
  ])
}
       
transform = image_transforms['test']

test_image_name = 'butterflies/train/ADONIS/001.jpg'
test_image = Image.open(test_image_name)
        
test_image_tensor = transform(test_image)
#print(model)
    
#device = torch.device('cpu')
#model = torch.load(model_path, map_location=device)
#print(model)
#summary(model, input_size=(3, 224, 224), batch_size=1, device='cpu')
    
layer = model._modules.get('avgpool')

my_embedding = torch.zeros(512 * 49)
#my_embedding = torch.zeros(2048)
    
def copy_data(m, i, o):
    my_embedding.copy_(o.flatten()) 
    
h = layer.register_forward_hook(copy_data)
    
with torch.no_grad():
    model(test_image_tensor.unsqueeze(0))
h.remove()
    
features = my_embedding.tolist()

len(features)

In [ ]:
import os
import glob
import json
import random
import numpy as np
from tqdm import tqdm
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [ ]:
def load_files(images_path, indexes):
  files = glob.glob(f'{images_path}/*/*.json')
  random.shuffle(files)
  labels = []
  features = []
  print('Loading files...')
  for f in tqdm(files):
    label = f.split('/')[-2]
    labels.append(label)
    with open(f, 'r') as features_file:
        features.append(np.array(json.load(features_file)))
        
  labels = np.array(labels)
  features = np.array(features)
  return features, labels

In [ ]:
dataset_path = 'butterflies'
train_directory = os.path.join(dataset_path, 'train')
valid_directory = os.path.join(dataset_path, 'valid')
test_directory = os.path.join(dataset_path, 'test')

In [ ]:
classes = os.listdir(valid_directory)
indexes = {v: k for k, v in enumerate(classes)}

x_train, y_train = load_files(train_directory, indexes)
x_valid, y_valid = load_files(valid_directory, indexes)

In [ ]:
print('Training classifier...')
clf = MLPClassifier(random_state=1, 
                    max_iter=300, 
                    verbose=True, 
                    shuffle=True, 
                    learning_rate_init=0.00001).fit(x_train, y_train)
print(clf.predict(x_valid))
print(clf.score(x_valid, y_valid))

In [ ]:
import numpy as np
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

clf = make_pipeline(StandardScaler(), SVC(gamma='auto', 
                    random_state=1, 
                    max_iter=300, 
                    verbose=True), verbose=True)
clf.fit(x_train, y_train)
print(clf.predict(x_valid))
print(clf.score(x_valid, y_valid))

In [ ]:
# Krinski#2891 Discord